Задание 1. 

Сгенерируйте последовательности цифр от 0 до 9, 
которые бы задавались следующим
образом:

● x - последовательность цифр

● y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

1. Научите модель предсказывать y(i) по x(i)
2. Попробуйте RNN, LSTM, GRU
Задание 2* (дополнительное и необязательное)
Примените LSTM для решения лекционного практического задания



In [225]:
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети
import numpy as np
import re



In [226]:
torch.manual_seed(42)
ds_digit_25 = torch.randint(0, 9, (2,1000,25))
ds_digit_75 = torch.randint(0, 9, (2,1000,75))
ds_digit_150 = torch.randint(0, 9, (2,1000,150))
ds_digit_25

tensor([[[6, 5, 7,  ..., 1, 5, 8],
         [4, 2, 0,  ..., 4, 0, 2],
         [3, 0, 1,  ..., 7, 7, 7],
         ...,
         [7, 6, 8,  ..., 8, 6, 4],
         [7, 8, 7,  ..., 5, 8, 7],
         [1, 4, 2,  ..., 1, 0, 0]],

        [[8, 8, 2,  ..., 2, 8, 1],
         [7, 4, 0,  ..., 4, 1, 1],
         [3, 3, 8,  ..., 3, 5, 4],
         ...,
         [1, 5, 8,  ..., 2, 7, 6],
         [6, 7, 2,  ..., 1, 4, 1],
         [4, 0, 5,  ..., 3, 2, 1]]])

In [227]:
ds_digit_25[0,:,:]

tensor([[6, 5, 7,  ..., 1, 5, 8],
        [4, 2, 0,  ..., 4, 0, 2],
        [3, 0, 1,  ..., 7, 7, 7],
        ...,
        [7, 6, 8,  ..., 8, 6, 4],
        [7, 8, 7,  ..., 5, 8, 7],
        [1, 4, 2,  ..., 1, 0, 0]])

In [228]:
vec_digit=torch.eye(10,10)
vec_digit

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [229]:
def f_cifer_cesar(dig_set):

    #dig_set[:][0][:] == X
    #dig_set[:][1][:] == Y

    for i in range(dig_set.shape[1]):
        for j in range(dig_set.shape[2]):
            if j==0:
                dig_set[1,i,j]=dig_set[0,i,j]
            else:
                dig_set[1,i,j]=dig_set[0,i,j]+dig_set[0,i,0]
                if dig_set[1,i,j]>9:
                    dig_set[1,i,j]=dig_set[1,i,j]-10
    return dig_set

In [230]:
ds_digit_25 = f_cifer_cesar(ds_digit_25)
ds_digit_75 = f_cifer_cesar(ds_digit_75)
ds_digit_150 = f_cifer_cesar(ds_digit_150)
ds_digit_25[0,0,:],ds_digit_25[1,0,:]

(tensor([6, 5, 7, 4, 0, 2, 7, 5, 4, 2, 4, 4, 8, 0, 0, 4, 2, 4, 3, 4, 4, 8, 1, 5,
         8]),
 tensor([6, 1, 3, 0, 6, 8, 3, 1, 0, 8, 0, 0, 4, 6, 6, 0, 8, 0, 9, 0, 0, 4, 7, 1,
         4]))

In [231]:
ds_digit_25[0,:,:].shape

torch.Size([1000, 25])

In [232]:
from sklearn.model_selection import  train_test_split
X_train_25, X_test_25, y_train_25, y_test_25 = train_test_split(ds_digit_25[0,:,:], ds_digit_25[1,:,:], test_size=0.2, random_state=42)

In [233]:
X_train_75, X_test_75, y_train_75, y_test_75 = train_test_split(ds_digit_75[0,:,:], ds_digit_75[1,:,:], test_size=0.2, random_state=42)

In [234]:
X_train_150, X_test_150, y_train_150, y_test_150 = train_test_split(ds_digit_150[0,:,:], ds_digit_150[1,:,:], test_size=0.2, random_state=42)

In [235]:
len(y_train_25),len(y_test_25)

(800, 200)

In [236]:

def input_data(seq_x, seq_y):
    out = []
    L = seq_x.shape[0]
    
    for i in range(L):
        x = seq_x[i,:]
        y = seq_y[i,:]
        out.append((x,y))
    
    return out

In [237]:

train_data_25 = input_data(X_train_25,y_train_25)
test_data_25 = input_data(X_test_25,y_test_25)

train_data_75 = input_data(X_train_75,y_train_75)
test_data_75 = input_data(X_test_75,y_test_75)

train_data_150 = input_data(X_train_150,y_train_150)
test_data_150 = input_data(X_test_150,y_test_150)


In [238]:
import torch.optim.lr_scheduler as lr_scheduler

In [239]:
class myGRU(torch.nn.Module):
    def __init__(self, rnnClass=torch.nn.GRU, dictionary_size=None, embedding_size=10, num_hiddens=128, num_classes=10):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        #self.embedding = torch.nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = torch.nn.Linear(num_hiddens, num_classes)
        
    def forward(self, seq):
        #out = self.embedding(X)
        _, state = self.hidden(seq.view(1,1,10))
        predictions = self.output(state[0].squeeze())
        return predictions
    
#model = NeuralNetwork(torch.nn.GRU, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [240]:
class myRNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = torch.nn.RNN(10, 128)
        self.out = torch.nn.Linear(128, 10)

    def forward(self, seq, state=None):
        #x = self.embedding(sentences)
        x, s = self.rnn(seq.view(1,1,10)) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)

In [241]:
class myLSTM(torch.nn.Module):
    
    def __init__(self,input_size = 10, hidden_size = 128, out_size = 10):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(input_size, hidden_size)
        self.linear = torch.nn.Linear(hidden_size,out_size)
        self.hidden = (torch.zeros(1,1,hidden_size),torch.zeros(1,1,hidden_size))
    
    def forward(self,seq):
        #lstm_out, self.hidden = self.lstm(seq.view(len(seq),1,-1), self.hidden)
        lstm_out, self.hidden = self.lstm(seq.view(1,1,10), self.hidden)
        pred = self.linear(lstm_out.view(128))
        return pred#[-1]

In [249]:
model = myGRU()
(type(model))

__main__.myGRU

In [250]:
def train_my_models(locmodel,train_data,test_data,epochs = 3):

    torch.manual_seed(42)
    
    criterion = torch.nn.MSELoss()#reduction='sum'
    optimizer = torch.optim.Adam(locmodel.parameters(), lr=0.001)
    scheduler1 = lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=0.00001)

    for i in range(epochs):
        
        for x_seq, y_seq in train_data[:80][:]:
            for x_seq_letter, y_seq_letter in zip(x_seq, y_seq):
                
                x_seq_letter=vectors_letter[x_seq_letter]
                y_seq_letter=vectors_letter[y_seq_letter]
                #print(x_seq_letter,'\n',y_seq_letter,'\n')

                optimizer.zero_grad()
                if type(locmodel) == myLSTM:
                    locmodel.hidden = (torch.zeros(1,1,locmodel.hidden_size),torch.zeros(1,1,locmodel.hidden_size))
                
                y_pred = locmodel(x_seq_letter)
                loss = criterion(y_pred.squeeze(), y_seq_letter)
                loss.backward()
                optimizer.step()
            
        
        scheduler1.step()
        
        for x_seq, y_seq in test_data[:20][:]:  
            for x_seq_letter, y_seq_letter in zip(x_seq, y_seq):  

                x_seq_letter=vectors_letter[x_seq_letter]
                y_seq_letter=vectors_letter[y_seq_letter]

                y_pred = locmodel(x_seq_letter.squeeze())
                
                loss_test = criterion(y_pred.squeeze(), y_seq_letter.squeeze())
                
        if (i+1) % 1 == 0:
            print(f"Epoch {i+1} MSE_TRAIN_Loss: {loss.item()}")
            print(f"          MSE_TEST__Loss: {loss_test}")

    return 0

In [251]:
train_my_models(myRNN(),train_data_25,test_data_25)
train_my_models(myRNN(),train_data_75,test_data_75)
train_my_models(myRNN(),train_data_150,test_data_150)

train_my_models(myGRU(),train_data_25,test_data_25)
train_my_models(myGRU(),train_data_75,test_data_75)
train_my_models(myGRU(),train_data_150,test_data_150)

train_my_models(myLSTM(),train_data_25,test_data_25)
train_my_models(myLSTM(),train_data_75,test_data_75)
train_my_models(myLSTM(),train_data_150,test_data_150)


Epoch 1 MSE_TRAIN_Loss: 0.09926282614469528
          MSE_TEST__Loss: 0.08214692771434784
Epoch 2 MSE_TRAIN_Loss: 0.09724553674459457
          MSE_TEST__Loss: 0.08198340237140656
Epoch 3 MSE_TRAIN_Loss: 0.09880097955465317
          MSE_TEST__Loss: 0.08440891653299332
Epoch 1 MSE_TRAIN_Loss: 0.04933413118124008
          MSE_TEST__Loss: 0.08907286822795868
Epoch 2 MSE_TRAIN_Loss: 0.053418297320604324
          MSE_TEST__Loss: 0.08606407046318054
Epoch 3 MSE_TRAIN_Loss: 0.062094878405332565
          MSE_TEST__Loss: 0.07785577327013016
Epoch 1 MSE_TRAIN_Loss: 0.01772923208773136
          MSE_TEST__Loss: 0.1257902830839157
Epoch 2 MSE_TRAIN_Loss: 0.02250424027442932
          MSE_TEST__Loss: 0.11995366960763931
Epoch 3 MSE_TRAIN_Loss: 0.03684030845761299
          MSE_TEST__Loss: 0.10945238173007965
Epoch 1 MSE_TRAIN_Loss: 0.10180799663066864
          MSE_TEST__Loss: 0.0846046656370163
Epoch 2 MSE_TRAIN_Loss: 0.10100273042917252
          MSE_TEST__Loss: 0.08549565821886063
Epoch 3 MS

0

In [252]:
def validation_mymodel(locmodel,test_data):
     i=0
     Accuracy=0
     
     if type(locmodel)==myRNN:
         model_name='myRNN'
     elif type(locmodel)==myGRU:
         model_name='myGRU'
     elif type(locmodel) == myLSTM:
         model_name='myLSTM'


     if test_data[:1][0][0].shape[0]==25:
          dataset_name='series_25'
     elif test_data[:1][0][0].shape[0]==75:
          dataset_name='series_75'
     elif test_data[:1][0][0].shape[0]==150:
          dataset_name='series_150'
             
     for x_seq, y_seq in test_data[:200][:]:  
            
            i+=1
            for x_seq_letter, y_seq_letter in zip(x_seq, y_seq):  

                x_seq_letter=vectors_letter[x_seq_letter]
                y_seq_letter=vectors_letter[y_seq_letter]

                y_pred = locmodel(x_seq_letter.squeeze())
                
                #loss_test = criterion(y_pred.squeeze(), y_seq_letter.squeeze())
                if torch.argmax(y_seq_letter)==torch.argmax(y_pred):
                    Accuracy+=1
                    
     print(f'Accuracy for {model_name} and {dataset_name}',Accuracy/(25*i))
     return Accuracy/(25*i)#model_name,dataset_name,

In [253]:
r,g,l=[],[],[]

r.append(validation_mymodel(myRNN(),test_data_25))
r.append(validation_mymodel(myRNN(),test_data_75))
r.append(validation_mymodel(myRNN(),test_data_150))

g.append(validation_mymodel(myGRU(),test_data_25))
g.append(validation_mymodel(myGRU(),test_data_75))
g.append(validation_mymodel(myGRU(),test_data_150))

l.append(validation_mymodel(myLSTM(),test_data_25))
l.append(validation_mymodel(myLSTM(),test_data_75))
l.append(validation_mymodel(myLSTM(),test_data_150))

Accuracy for myRNN and series_25 0.1112
Accuracy for myRNN and series_75 0.3314
Accuracy for myRNN and series_150 0.5924
Accuracy for myGRU and series_25 0.0924
Accuracy for myGRU and series_75 0.2742
Accuracy for myGRU and series_150 0.559
Accuracy for myLSTM and series_25 0.1072
Accuracy for myLSTM and series_75 0.3058
Accuracy for myLSTM and series_150 0.6056


In [254]:
from prettytable import PrettyTable
x = PrettyTable()

x.field_names = ['LenghtOfSeries',"RNN", "GRU", "LSTM"]
x.add_row(['series_25 ',r[0], g[0], l[0] ])
x.add_row(['series_75 ',r[1], g[1], l[1]])
x.add_row(['series_150',r[2], g[2], l[2]])

print(x)

+----------------+--------+--------+--------+
| LenghtOfSeries |  RNN   |  GRU   |  LSTM  |
+----------------+--------+--------+--------+
|   series_25    | 0.1112 | 0.0924 | 0.1072 |
|   series_75    | 0.3314 | 0.2742 | 0.3058 |
|   series_150   | 0.5924 | 0.559  | 0.6056 |
+----------------+--------+--------+--------+


Задание 2:
применить LSTM для решения лекционного практического задания

In [2]:
import torch
from torch import nn
import re
import random
import tqdm
import time

# !wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

In [3]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [4]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [5]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [6]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [7]:
BATCH_SIZE=40
dataset = torch.utils.data.TensorDataset(X[:-35], Y[:-35])
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X.squeeze())
        out, state = self.hidden(out)
        predictions = self.output(out[0].squeeze())
        return predictions

In [9]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [10]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred)#[0] #model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [14]:
generate_text()

old it addresses itself to men free from|frroo slvt  rnt asrceshsur h nshduetsdag


In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [13]:
for ep in range(2):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in data:
        #X_b, y_b = X_b, y_b#X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model(X_b.squeeze())
        loss = criterion(answers.squeeze(), y_b.squeeze())
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

Epoch 0. Time: 257.145, Train loss: 2.845
 ensues accompanied by a desire for some|e tmii otpn tm rpdtnec to ottnet ibeeect
Epoch 1. Time: 268.921, Train loss: 2.839
ism in europe let us not be ungrateful t|stert i  t nghiaennki  pod sasw   ofte l
